## 데이터 로드

In [1]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [2]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date', 'like_cnt'], axis=1)
train_data.head()

,tags,songs
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [3]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [4]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [5]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [6]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [7]:
train_data_sample = train_data[:2000]

## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [8]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [9]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'걸그룹댄스', 'kpop', '댄스', '걸그룹', '스트레스해소', '꽃놀이', '회상', '기분전환', '추억', '다이어트', '고백', '봄', '잔잔한', '시작', '사랑', '최신', '신나는노래', '여자아이돌', '월요일', '일상', '아이돌', '히트곡'}


In [10]:
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

(52771, 2)

In [11]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [12]:
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,[락],525514,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,[락],129701,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,[락],383374,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,[락],562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,[락],297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


In [13]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52771 entries, 0 to 52770
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                52771 non-null  object
 1   song_id             52771 non-null  int64 
 2   gnr                 52771 non-null  object
 3   issue_date          52771 non-null  int64 
 4   album_id            52771 non-null  int64 
 5   artist_id_basket    52771 non-null  object
 6   song_name           52771 non-null  object
 7   artist_name_basket  52771 non-null  object
dtypes: int64(3), object(5)
memory usage: 3.6+ MB


## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추
- 2000개의 플레이리스트를 활용

In [14]:
train_data_sample2 = train_data[:2000]

In [15]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = train_data_sample2['tags'], vector_size = 100, 
               window = 5, min_count = 5, workers = 4, sg = 1)

w2v.wv.vectors.shape

(214, 100)

In [16]:
print(w2v.wv.most_similar('스트레스'))

[('커피', 0.9954237341880798), ('잔잔한', 0.9954167008399963), ('드라이브', 0.9952830076217651), ('취향저격', 0.9952302575111389), ('기분전환', 0.9951197504997253), ('사랑', 0.9950689077377319), ('여행', 0.9950666427612305), ('카페', 0.9949964284896851), ('그루브', 0.9948758482933044), ('퇴근길', 0.9948517084121704)]


## 코사인 유사도 사용

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer()
gnr_mat = count_vect.fit_transform(song_tag_appended['gnr_literal'])

gnr_mat.shape

(52771, 209)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.33333333, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.57735027,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.57735027, 1.        ,
        0.57735027],
       [0.        , 0.        , 0.        , ..., 1.        , 0.57735027,
        1.        ]])

In [19]:
simi_test = gnr_sim[1, :]
simi_test2 = gnr_sim[2, :]
print(simi_test + simi_test2)
print(song_data[song_data['song_id']==525514])
print(song_data[song_data['song_id']==129701])
print(song_data[song_data['song_id']==229622])

[0.         1.33333333 1.33333333 ... 0.         0.         0.        ]
                     gnr  issue_date  album_id artist_id_basket  \
525514  [GN1402, GN1401]    20130506   2200223         [734201]   

              song_name artist_name_basket  song_id  
525514  Hey Little Girl          [The Sol]   525514  
                             gnr  issue_date  album_id artist_id_basket  \
129701  [GN0901, GN0902, GN1001]    20130917   2201802         [536907]   

       song_name artist_name_basket  song_id  
129701   Octagon      [Royal Bangs]   129701  
                                     gnr  issue_date  album_id  \
229622  [GN1402, GN1401, GN0901, GN0902]    20061212    528351   

       artist_id_basket  song_name artist_name_basket  song_id  
229622         [406781]  Say Hello     [Rosie Thomas]   229622  


- 노래 id가 주어지면 유사도 순으로 n개의 노래 추출

In [20]:
import numpy as np

def find_sim_song(df, sim_matrix, songs, top_n=10):
    simi = np.zeros(len(df['song_id']))
    minyear = 3000
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        minyear = min(minyear, title_song['issue_date'].values[0]//10000)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
        simi = simi + sim_matrix[title_index, :]
    
    simi /= len(songs)
    
    df['similarity'] = simi.reshape(-1, 1)
    temp = df.sort_values(by="similarity", ascending=False)
    
    #for song in songs:
    #    title_song = df[df['song_id'] == song]
    #    title_index = title_song.index.values
    #    
    #    temp = temp[temp.index.values != title_index]
    
    temp = temp[temp['issue_date'] > minyear*10000]
    
    # 유사도가 0.5 이하인 경우는 제외
    temp = temp[temp['similarity'] >= 0.5]
    
    temp = temp.reset_index(drop=True)
    
    #final_index = temp.index.values[ : top_n]
    
    return temp.iloc[ : top_n]

In [21]:
similar_songs = find_sim_song(song_tag_appended, gnr_sim, [525514, 129701, 229622], 10)

similar_songs[['song_id', 'similarity', 'issue_date', 'gnr']]

,song_id,similarity,issue_date,gnr
0,284591,0.767160,20111003,"[GN1402, GN1401, GN0901, GN0902, GN1001]"
1,110099,0.767160,20111003,"[GN1402, GN1401, GN0901, GN0902, GN1001]"
2,695590,0.761486,20141103,"[GN1402, GN1401, GN0901, GN0902]"
3,229622,0.761486,20061212,"[GN1402, GN1401, GN0901, GN0902]"
4,191430,0.761486,20140407,"[GN1402, GN1401, GN0901, GN0902]"
5,463782,0.761486,20170227,"[GN1402, GN1401, GN0901, GN0902]"
6,343152,0.761486,20141103,"[GN1402, GN1401, GN0901, GN0902]"
7,167737,0.761486,20090630,"[GN1402, GN1401, GN0901, GN0902]"
8,661051,0.700318,20090721,"[GN0901, GN0902, GN1402, GN1401, GN1013, GN1001]"
9,105967,0.671952,20110502,"[GN1402, GN1401, GN0901]"


## 노래 추천

- w2v로 추출한 태그에 해당하는 플레이리스트
- 세부 장르의 유사도가 높은 노래 리스트
- 히스토리(test 플레이리스트)의 발행 연도와 같은 연도에 발행한 노래

In [22]:
def song_recommend(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
        print(simi_songs)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
        
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        
        # 태그와 유사도 두 부분에서 동일한 개수(홀수일 경우 태그 > 유사도) 추출
        # sc = 유사도 부분에서 추출할 노래의 개수
        if len(recommended) % 2 == 0:
            sc = (10-len(recommended)) / 2
        else:
            sc = (10-len(recommended)) // 2
        
        # 태그는 있고 히스토리가 없는 경우 태그 부분에서 10개를 추출하기 위한 코드
        if len(songs) == 0:
            sc = 0
        
        # 이미 추출한 노래를 제외하고 태그 부분에서 정해진 개수만큼 추출한다
        # 태그가 없을 경우 동작하지 않음
        index = 0
        while len(tag_songs) != 0 and len(recommended) < (10 - sc):
            tag_song = tag_songs[index][0]
            
            if not tag_song in recommended:
                recommended.append(tag_song)
            
            index += 1
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 부분에서 추출한다
        index = 0
        while len(recommended) < 10:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]

In [23]:
def song_recommend_test(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
    
    
    
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 부분에서 추출한다
        index = 0
        while len(recommended) < 10:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
        if len(recommended) < 10:
            index = 0
            while len(tag_songs) != 0 and len(recommended) < 10:
                tag_song = tag_songs[index][0]
                if not tag_song in recommended:
                    recommended.append(tag_song)
                index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]

### 태그와 노래 목록 존재

In [24]:
my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

                                                 tags  song_id  \
0                                        [회상, 추억, 까페]   284591   
1                                        [회상, 추억, 까페]   110099   
2   [퇴폐적, 우울, 몽환적인, 힐링, 몽환적, 우울함, 휴식, 밤, 새벽, 우주, 밤...   695590   
3             [커플, 드라이브, 감성, 휴식, 잔잔한, 겨울, 설렘, 사랑, 차안]   229622   
4                                       [휴식, 잔잔한, 힐링]   191430   
..                                                ...      ...   
95                                                [팝]   289617   
96                                                [팝]   153498   
97                           [FM4U, 200312, 라디오, MBC]   197175   
98                                       [휴식, 까페, 힐링]    65115   
99                                [잔잔한, 휴식, 감성, 드라이브]   292093   

                                         gnr  issue_date  album_id  \
0   [GN1402, GN1401, GN0901, GN0902, GN1001]    20111003   2017388   
1   [GN1402, GN1401, GN0901, GN0902, GN1001]    20111003   2017388 

,song_id,song_name,artist_name_basket,issue_date,tags
11922,592021,How To Save a Life,[The Fray],20050913,"[PopRock, 팝락, 가을, 밤, 기타, 기분전환, Rock, 얼터너티브, 우울..."
11923,317882,Be Be Your Love,[Rachael Yamagata],20070119,"[스트레스, 락, 잔잔한, 밤, 기분전환, 비오는날, 인디, 팝, 새벽]"
3099,517064,Haven`t Met You Yet,[Michael Buble],20091009,"[여행, 댄스, 휴식, 알앤비, 까페, 소울, 매장, 드라이브, 신나는, 기분전환,..."
3588,143163,She Will Be Loved,[Maroon 5],20111114,"[파티, 여행, 드라이브, 중독성, 주말, 댄스, 휴식, 락, 봄, 신나는, 기분전..."
12373,101388,바래,[FTISLAND (FT아일랜드)],20090716,"[댄스곡, 스트레스, 보이그룹, 회상, 댄스, 휴식, 락, 걸그룹, 내적댄스, 신나..."
50753,284591,Black Flies,[Ben Howard],20111003,"[회상, 추억, 까페]"
50754,110099,Promise,[Ben Howard],20111003,"[회상, 추억, 까페]"
29469,695590,It Takes A Lot To Know A Man,[Damien Rice],20141103,"[퇴폐적, 우울, 몽환적인, 힐링, 몽환적, 우울함, 휴식, 밤, 새벽, 우주, 밤..."
4912,229622,Say Hello,[Rosie Thomas],20061212,"[커플, 드라이브, 감성, 휴식, 잔잔한, 겨울, 설렘, 사랑, 차안]"
11005,191430,Sing,[Ed Sheeran],20140407,"[휴식, 잔잔한, 힐링]"


### 태그와 목록 존재
- song_recommend_test 함수로 변경 (태그와 장르 교집합 비교 -> 장르에서 추출 -> 태그에서 추출 순서로 우선순위 부여)

In [25]:
my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend_test(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
50753,284591,Black Flies,[Ben Howard],20111003,"[회상, 추억, 까페]"
50754,110099,Promise,[Ben Howard],20111003,"[회상, 추억, 까페]"
29469,695590,It Takes A Lot To Know A Man,[Damien Rice],20141103,"[퇴폐적, 우울, 몽환적인, 힐링, 몽환적, 우울함, 휴식, 밤, 새벽, 우주, 밤..."
4912,229622,Say Hello,[Rosie Thomas],20061212,"[커플, 드라이브, 감성, 휴식, 잔잔한, 겨울, 설렘, 사랑, 차안]"
11005,191430,Sing,[Ed Sheeran],20140407,"[휴식, 잔잔한, 힐링]"
15445,463782,Dreamin` Slow,[Mac Demarco],20170227,"[혼자, 주말, 휴식, 잔잔한, 분위기, 책읽을때, 휴가, 추억, 감성적인]"
29470,343152,The Box,[Damien Rice],20141103,"[밤, 휴식, 새벽, 힐링]"
42332,167737,You Got Me,[Colbie Caillat],20090630,[팝]
48415,661051,Santeria,[Aimee Allen],20090721,"[기분전환, 스트레스]"
48408,105967,Offbeat,[Clara C (클라라C)],20110502,"[기분전환, 스트레스]"


### 노래 목록만 존재

In [26]:
my_tags2 = []
my_songs2 = [525514, 129701, 229622]
rec2 = song_recommend(my_tags2, my_songs2, train_data_sample2, song_tag_appended, gnr_sim)
rec2[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

                                                 tags  song_id  \
0                                        [회상, 추억, 까페]   284591   
1                                        [회상, 추억, 까페]   110099   
2   [퇴폐적, 우울, 몽환적인, 힐링, 몽환적, 우울함, 휴식, 밤, 새벽, 우주, 밤...   695590   
3             [커플, 드라이브, 감성, 휴식, 잔잔한, 겨울, 설렘, 사랑, 차안]   229622   
4                                       [휴식, 잔잔한, 힐링]   191430   
..                                                ...      ...   
95                                                [팝]   289617   
96                                                [팝]   153498   
97                           [FM4U, 200312, 라디오, MBC]   197175   
98                                       [휴식, 까페, 힐링]    65115   
99                                [잔잔한, 휴식, 감성, 드라이브]   292093   

                                         gnr  issue_date  album_id  \
0   [GN1402, GN1401, GN0901, GN0902, GN1001]    20111003   2017388   
1   [GN1402, GN1401, GN0901, GN0902, GN1001]    20111003   2017388 

,song_id,song_name,artist_name_basket,issue_date,tags
50753,284591,Black Flies,[Ben Howard],20111003,"[회상, 추억, 까페]"
50754,110099,Promise,[Ben Howard],20111003,"[회상, 추억, 까페]"
29469,695590,It Takes A Lot To Know A Man,[Damien Rice],20141103,"[퇴폐적, 우울, 몽환적인, 힐링, 몽환적, 우울함, 휴식, 밤, 새벽, 우주, 밤..."
4912,229622,Say Hello,[Rosie Thomas],20061212,"[커플, 드라이브, 감성, 휴식, 잔잔한, 겨울, 설렘, 사랑, 차안]"
11005,191430,Sing,[Ed Sheeran],20140407,"[휴식, 잔잔한, 힐링]"
15445,463782,Dreamin` Slow,[Mac Demarco],20170227,"[혼자, 주말, 휴식, 잔잔한, 분위기, 책읽을때, 휴가, 추억, 감성적인]"
29470,343152,The Box,[Damien Rice],20141103,"[밤, 휴식, 새벽, 힐링]"
42332,167737,You Got Me,[Colbie Caillat],20090630,[팝]
48415,661051,Santeria,[Aimee Allen],20090721,"[기분전환, 스트레스]"
48408,105967,Offbeat,[Clara C (클라라C)],20110502,"[기분전환, 스트레스]"


### 태그만 존재

In [27]:
my_tags3 = ['락']
my_songs3 = []
rec3 = song_recommend(my_tags3, my_songs3, train_data_sample2, song_tag_appended, gnr_sim)
rec3[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
11922,592021,How To Save a Life,[The Fray],20050913,"[PopRock, 팝락, 가을, 밤, 기타, 기분전환, Rock, 얼터너티브, 우울..."
11923,317882,Be Be Your Love,[Rachael Yamagata],20070119,"[스트레스, 락, 잔잔한, 밤, 기분전환, 비오는날, 인디, 팝, 새벽]"
3099,517064,Haven`t Met You Yet,[Michael Buble],20091009,"[여행, 댄스, 휴식, 알앤비, 까페, 소울, 매장, 드라이브, 신나는, 기분전환,..."
3588,143163,She Will Be Loved,[Maroon 5],20111114,"[파티, 여행, 드라이브, 중독성, 주말, 댄스, 휴식, 락, 봄, 신나는, 기분전..."
12373,101388,바래,[FTISLAND (FT아일랜드)],20090716,"[댄스곡, 스트레스, 보이그룹, 회상, 댄스, 휴식, 락, 걸그룹, 내적댄스, 신나..."
12375,354259,사랑이란건,[부활],20100322,"[락, 이별, 비오는날, 발라드, 슬픔]"
21483,460936,Don&#39;t Stop Me Now (2011 Mix),[Queen],20031201,[락]
22723,417666,If I Die Young,[The Band Perry],20101011,[락]
5540,62200,Bad Day,[Daniel Powter],20050408,"[위스키, 드라이브, 휴식, 락, 잔잔한, 피아노, 밤, 기타, 와인, 록, 일렉,..."
5614,652798,Back Down To Earth,[Michelle Shaprow],20110111,"[여행, 휴식, 리스트, 스트레스해소, 소풍, 회상, 감성, 드라이브, 신나는, 밤..."


### 둘 다 없음

In [28]:
my_tags4 = []
my_songs4 = []
rec4 = song_recommend(my_tags4, my_songs4, train_data_sample2, song_tag_appended, gnr_sim)
rec4[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
89,394031,Into the Unknown (From &#34;Frozen 2&#34;/Soun...,"[Idina Menzel, Aurora]",20191115,"[크리스마스송, 세벽감성, 한번쯤들어봤던노래, 크리스마스, 트로트, 유명한, 스트레..."
119,457519,꿀차,[우효],20180102,"[크리스마스송, 휴식, 크리스마스, 위로, 감성, 겨울왕국, 기분전환, 비오는날, ..."
120,453762,너 정말 예쁘다,[최낙타],20180410,"[크리스마스송, 여행, 어쿠스틱, 크리스마스, 썸, 포크, 겨울왕국, 이갈룡, 20..."
121,349398,LOVE YA!,[혁오 (HYUKOH)],20180531,"[크리스마스송, 캐럴, 따듯한, 겨울노래, 감성, 겨울왕국, 일상속, 드라이브, 크..."
122,631142,편지,[장희원],20180603,"[크리스마스송, 크리스마스, 새벽, 겨울왕국, 감성자극, 밤, 비오는날, 달콤한, ..."
123,406082,하늘엔 별이 떠있고 너만큼은 빛나질 않아,[이민혁],20180903,"[크리스마스송, 캐럴, 따듯한, 겨울노래, 겨울왕국, 크리스마스캐럴, 분위기, 눈오..."
124,548389,사랑에 빠졌네,[정준일],20181101,"[크리스마스송, 캐럴, 따듯한, 겨울노래, 겨울왕국, 크리스마스캐럴, 분위기, 눈오..."
125,205179,꿀맛,[정미애],20190815,"[12월, 연말노래, 두근두근, 여성보컬, 가을, 그루브, 연말감성, 고속도로, 감..."
126,567076,숨겨진 세상 (Into the Unknown End Credit Version) (...,[태연 (TAEYEON)],20191115,"[크리스마스송, 세벽감성, 한번쯤들어봤던노래, 저녁, 크리스마스, 퇴근길, 유명한,..."
180,418694,Attention,[Charlie Puth],20180511,"[음색깡패, 여행, 휴식, 낙엽, 느낌있는, 듣기좋은, 내꺼, 트렌드, 가을, 그루..."


# 모델 평가
- [참고](https://chrisjune-13837.medium.com/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C-%EC%84%B1%EB%8A%A5%ED%8F%89%EA%B0%80%EB%B0%A9%EB%B2%95-with-python-9932097f0ff9)

# Recall@K

In [29]:
def get_recall_k(y_true, y_pred):
    recall_k = 0
    
    true_items = set(y_true)
    pred_items = set(y_pred)
    intersect_items = len(true_items.intersection(pred_items))
    recall = intersect_items / len(true_items) if len(true_items) > 0 else 0
    return recall

my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend_test(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]
pred_list = rec1['song_id'].tolist()

recall_k = get_recall_k(my_songs1, pred_list)
print("Recall@K (K=10): {:.2f}".format(recall_k))

Recall@K (K=10): 0.33


# Precision@K

In [30]:
def get_precision_k(y_true, y_pred):
    precision_k = 0
    
    true_items = set(y_true)
    pred_items = set(y_pred)
    intersect_items = len(true_items.intersection(pred_items))
    recall = intersect_items / len(pred_items) if len(pred_items) > 0 else 0
    return recall

my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend_test(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]
pred_list = rec1['song_id'].tolist()

precision_k = get_precision_k(my_songs1, pred_list)
print("Precision@K (K=10): {:.2f}".format(precision_k))

Precision@K (K=10): 0.10


# nDCG
- Relevance Score를 저장하는 리스트 생성
- 만약 기존 플레이리스트에 있었다면 1, 그렇지 않으면 0.1을 리스트에 저장
- 그 후 이 리스트를 내림차순으로 정렬한 또다른 리스트 생성
- 이 둘을 비교해 nDCG 도출

In [31]:
def ndcg_at_k(ranking, k):
    # Ideal ranking을 계산하기 위해 ranking을 내림차순으로 정렬한 별도의 리스트
    ideal_ranking = sorted(ranking, reverse=True)
    
    # DCG 계산
    dcg = ranking[0]
    for i in range(1, min(k, len(ranking))):
        dcg += ranking[i] / np.log2(i + 1)
    
    # Ideal DCG 계산
    ideal_dcg = ideal_ranking[0]
    for i in range(1, min(k, len(ideal_ranking))):
        ideal_dcg += ideal_ranking[i] / np.log2(i + 1)
    
    # nDCG 계산
    if ideal_dcg == 0:
        ndcg = 0
    else:
        ndcg = dcg / ideal_dcg
    
    return ndcg

my_tags_ndcg = ['락']
my_songs_ndcg = [525514, 129701, 229622]
rec_ndcg = song_recommend_test(my_tags_ndcg, my_songs_ndcg, train_data_sample2, song_tag_appended, gnr_sim)
rec_ndcg[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]
pred_list = rec_ndcg['song_id'].tolist()

ranking_data = []
k = 10

for i in range(k):
    if pred_list[i] in my_songs_ndcg:
        ranking_data.append(1)
    else:
        ranking_data.append(0.1)
        
result = ndcg_at_k(ranking_data, k)
print(f'nDCG@{k}: {result:.4f}')

nDCG@10: 0.6843


# train 데이터 중에 랜덤으로 뽑아 테스트하기

In [32]:
my_tags_test_list = train_data['tags'].tolist()
my_songs_test_list = train_data['song_id'].tolist()
random_num = [342, 119, 900, 394, 5, 688, 142, 423, 59, 751]
total_ndcg = 0

for i in random_num:
    my_tags_test = my_tags_test_list[i]
    my_songs_test = my_songs_test_list[i]
    print(my_tags_test)
    print(my_songs_test)
    rec_test = song_recommend_test(my_tags_test, my_songs_test, train_data_sample2, song_tag_appended, gnr_sim)
    rec_test[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]
    pred_list = rec_test['song_id'].tolist()

    ranking_data = []
    k = 10

    for j in range(k):
        if pred_list[j] in my_songs_test:
            ranking_data.append(1)
        else:
            ranking_data.append(0.1)
            
    ndcg = ndcg_at_k(ranking_data, k)
    total_ndcg += ndcg
    print(f'nDCG@{k} of {i}th Playlist: {ndcg:.4f}')
    
avg_ndcg = total_ndcg / len(random_num)
print(f'Average nDCG@10: {avg_ndcg:.4f}')

['겨울', '밤', '새벽']
[430438, 143843, 529942, 40087, 679259, 630498, 533554, 556447, 507072, 255182, 326424, 622548, 653028, 77226, 259818, 30285, 519917, 305708, 53478, 531528, 112971, 658434, 634998, 174416, 61159, 676323, 449836, 568608, 22242, 569193, 40919, 651057, 599123, 89807, 343991, 419807, 216783, 268174, 239352, 657882, 498846, 190473]


IndexError: index 0 is out of bounds for axis 0 with size 0

# val.json 데이터 로드(이 밑으로는 일단 보류)

In [ ]:
with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
    
val_data = pd.DataFrame(json_data)
val_data = val_data.drop(['id', 'plylst_title', 'updt_date', 'like_cnt'], axis=1)
val_data.head()

In [ ]:
my_tags_test_list = val_data['tags'].tolist()
my_songs_test_list = val_data['songs'].tolist()
pred_list_test = []
result_test = []

In [ ]:
for i in range(len(val_data)):
    my_tags_test = my_tags_test_list[i]
    my_songs_test = my_songs_test_list[i]
    rec_test = song_recommend_test(my_tags_test, my_songs_test, train_data_sample2, song_tag_appended, gnr_sim)
    rec_test[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]
    pred_list= rec_test['song_id'].tolist()

    ranking_data = []
    k = 10

    for j in range(k):
        if pred_list[j] in my_songs_ndcg:
            ranking_data.append(1)
        else:
            ranking_data.append(0.1)
            
    result_test.append(ndcg_at_k(ranking_data, k))
    

In [ ]:
result_final = sum(result_test) / len(result_test)
print(f'nDCG@{k}: {result:.4f}')